지금까지 배운 것을 정리하면, 아래의 4단계를 거쳐 학습이 이루어진다.

1. 미내배치 선정
2. 기울기 산출
3. 매개변수 갱신 (가중치)
4. 반복 (1 ~ 3 단계)

이 때 미니배치를 무작위로 선정하는 방법을 두고, 확률적 경사 하강법이라고 부른다. 머리글자를 따서 SGD라고도 한다. 그럼 실제로 손글씨 숫자를 학습하는 신경망을 구현해보자.

# 4.5.1 2층 신경망 클래스 구현하기

In [3]:
import sys, os
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size,
                 weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        z2 = sigmoid(a2)

        return z2

    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

<p align="center"><img src="imgs/4-10s.png" width=500></p>

In [5]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
print(net.params['W1'].shape)
print(net.params['b1'].shape)
print(net.params['W2'].shape)
print(net.params['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)


위와 같이 params 변수에 신경망에 필요한 가중치를 모두 저장한다. 책에서는 매개변수라고 표현하는데, 가중치랑 동일한 의미이다. 여기서 가중치에는 바이어스도 포함된다. 이러한 가중치들은 순방향 계산에 이용된다. 참고로 예측 처리는 다음과 같이 실행할 수 있다.

In [9]:
import numpy as np

x = np.random.rand(100, 784)
y = net.predict(x)
y.shape

(100, 10)

In [11]:
x = np.random.rand(100, 784)
t = np.random.rand(100, 10)

grads = net.numerical_gradient(x, t)

print(grads['W1'].shape)
print(grads['b1'].shape)
print(grads['W2'].shape)
print(grads['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)


위의 계산은 numerical_gradient를 사용하는데, 시간이 꽤 오래 걸린다. 이와 유사하지만 거의 비슷한 계산을 하는 함수가 gradient 함수인데, 이를 이해하는 것에 중점을 두면 좋을 것 같다.

# 4.5.2 미니배치 학습 구현하기

In [6]:
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

(train_x, train_y), (test_x, test_y) = load_mnist(
    normalize=True, one_hot_label=True
)

train_loss_list = []

# hyper-parameters
iter_num = 10
train_size = train_x.shape[0]
batch_size = 100
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iter_num):
    print(i)
    # 미니배치 가져오기
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = train_x[batch_mask]
    y_batch = train_y[batch_mask]

    # 기울기 계산
    grad = network.numerical_gradient(x_batch, y_batch)

    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    # 학습 경과 기록
    loss = network.loss(x_batch, y_batch)
    train_loss_list.append(loss)

0
1
2
3
4
5
6
7
8
9


너무 오래 걸려서 포기... 일단은 10에폭만 돌렸다. 그래프르르 그려보면 아래와 같이 그려진다고 한다.

<p align="center"><img src="imgs/4-11.png" width=600></p>

# 4.5.3 시험 데이터로 평가하기

위의 그래프에서 잠깐 학습을 진행했다. 그래프를 잘 보면, 손실함수의 값이 점점 줄어드는 것을 확인할 수 있다. 이는 신경망이 잘 학습되고 있다는 말이지만, 이 모델을 전체 데이터에 적용할 수 있는 지는 또 다른 문제이다. 이를 모델의 일반화 문제라고 하는데, 학습이 잘 되었을지라도 이것이 현재 데이터에만 너무 맞춰져 학습이 되지 않았는지 확인하는 절차가 필요하다. 현재 데이터에만 너무 피팅된 것은 오버피팅이라고 하는데, 딥러닝 학습에서 이 오버피팅 문제를 항상 잘 해결해야 한다. 먼저, 오버피팅이 발생했는지 확인해는 것부터 해보자. 이는 TEST Data로 평가하는 방식을 통해 알아낼 수 있다.


In [2]:
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

(train_x, train_y), (test_x, test_y) = load_mnist(
    normalize=True, one_hot_label=True
)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# 하이퍼파라미터
iter_num = 10000
train_size = train_x.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1 에폭 당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)
print('iter per epoch:', iter_per_epoch)

for i in range(iter_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = train_x[batch_mask]
    t_batch = train_x[batch_mask]

    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    # grad = network.gradient(x_batch, t_batch)

    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    # 학습 경과 기록
    loss = network.loss(x_batch, y_batch)
    train_loss_list.append(loss)

    # 1 에폭당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(train_x, train_y)
        test_acc = network.accuracy(test_x, test_y)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)

        print('train_acc, test_acc  | ' + str(train_acc) + ',' + str(test_acc))

iter per epoch: 600.0


IndexError: arrays used as indices must be of integer (or boolean) type